In [7]:
import pandas as pd
import requests
import json
import sqlite3
import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import re as re
import seaborn as sns
from ast import literal_eval

# Anilist GraphQL API call:

The following functions will request data from https://anilist.co. If you would like to explore additional arguments and field reesponses, you can use the sandbox at https://studio.apollographql.com/sandbox/explorer, and enter https://graphql.anilist.co as the sandbox in the top right corner.

In [8]:
#Function to query anilist.co for a page from thier anime database, excluding explicit content and including only anime with the FINISHED status.
#It also creates a useable data frame for the query.
def anilist_query(i) -> pd.DataFrame:
  query= '''
  query Query($type: MediaType, $status: MediaStatus, $page: Int, $isAdult: Boolean) {
    Page(page: $page) {
      pageInfo {
        currentPage
        hasNextPage
      }
      media(type: $type, status: $status, isAdult: $isAdult) {
        id
        idMal
        title {
          english
          romaji
        }
        type
        format
        status
        seasonYear
        source
        genres
        popularity
        tags {
          name
        }
      }
    }
  }
  ''' 
  variables={
    "type" : 'ANIME',
    "status": 'FINISHED',
    "page" : i,
    "isAdult" : False
  }
  response = requests.post(url, json={'query': query,'variables': variables}) # Make the HTTP Api request
  json = response.json()
  df = pd.json_normalize(json['data']) #parse json for data column
  return df

In [9]:
# Function to loop all pages and append to data frame
def anilist_compile(i) -> pd.DataFrame:
    result = pd.DataFrame() #Create empty data frame.
    while True:
        df = anilist_query(i)
        if df['Page.pageInfo.hasNextPage'][0] == np.True_: #test for if there is another page to query
            df_1 = pd.json_normalize(df['Page.media']).T #flaten db for Page.media and transform
            df_2 = pd.json_normalize(df_1[0])  #flaten db for concatinate
            result = pd.concat([result,df_2], axis=0) #concatinate for result
            i += 1
            time.sleep(3) #limit request rate to complie with anilist
        else:
            df_1 = pd.json_normalize(df['Page.media']).T #flaten db for Page.media and transform
            df_2 = pd.json_normalize(df_1[0]) #flaten db for concatinate
            result = pd.concat([result,df_2], axis=0) #concatinate for result
            break
    return result

In [10]:
url = 'https://graphql.anilist.co'

In [ ]:
anilist = anilist_compile(1) #compiling from page 1 will ensure all titles are included

In [ ]:
anilist.to_csv("complete_anilist.csv",index=False) # Write to csv.

# User Data Collection:
The following funcition will return a users completed anime list from https://anilist.co. You can return your own list by changing the username in mylist.

In [ ]:
# Function to query anilist.co for a defined user's anime media list with the completed status, and transform it into a useable data frame.
def my_anilist(user_name) -> pd.DataFrame:
    query = '''
    query Query($userName: String, $type: MediaType, $status: MediaListStatus) {
    MediaListCollection(userName: $userName, type: $type, status: $status) {
        lists {
        entries {
            id
            mediaId
            media {
            title {
                english
                romaji
            }
            format
            }
        }
        name
        }
    }
    }
        '''
    variables={
        "type" : 'ANIME',
        "status": 'COMPLETED',
        "userName" : user_name,
    }
    response = requests.post(url, json={'query': query,'variables': variables}) # Make the HTTP Api request
    json = response.json()
    df = pd.json_normalize(json['data']) #parse json for data column
    df = pd.json_normalize(df['MediaListCollection.lists']) 
    df =pd.json_normalize(df[0])
    df = pd.json_normalize(df['entries']).T #transform data frame to be readable
    df = pd.json_normalize(df[0])
    df = df[(df['media.format'] != "MOVIE") & (df['media.format'] != "SPECIAL") & (df['media.format'] != "MUSIC") & (df['media.format'] != "TV_SHORT")] #remove unnecesary formats
    df['media.title.english'] = df['media.title.english'].fillna(df['media.title.romaji'])  #clean english title
    df = df.drop(['media.title.romaji','id','media.format'],axis=1) #drop romaji title, userid, and the format of the media
    return df

In [ ]:
my_list = my_anilist("leolion023") #change username to import your personal list
my_list

In [ ]:
my_list.to_csv('my_anilist.csv', index=False) #Write to csv

# My Anime List data for Mahou Shoujo

In anilist_analysis.ipynb we do a short section comparing user popularity data from our target data set from anilist.co to the equivalent set of data from My Anime List, a site with a larger user base but a worse API experience. To accomplish this we need a short list of My Anime List ids to be able to run their API. Luckily anilist's API will also return the id for MAL. First we will import the anilist data from the csv so we don't need to run anilist_compile every time we make a query to MAL API.

In [ ]:
df = pd.read_csv('complete_anilist.csv')

In [ ]:
df

In [ ]:
df_2 = df[(df.format != "MOVIE") & (df.format != "SPECIAL") & (df.format != "MUSIC") & (df.format != "TV_SHORT") & (df.format.notnull())] #drop irrelivant format
df_2['title.english'] = df['title.english'].fillna(df['title.romaji']) #fill missing english titles whit romaji
df_2 = df_2.dropna(subset=['seasonYear','idMal']) #drop missing year and mal id data
df_2 = df_2.drop(['title.romaji','status','type','format','source','tags'],axis=1) # drop irrelivant columns
df_2['seasonYear'] = df_2['seasonYear'].astype(int) #change float to int
df_2.rename(columns={'title.english' : 'Title'}, inplace=True) #rename columns to make more readable
df_2.rename(columns={'seasonYear' : 'Year'}, inplace=True) #rename column
df_3 = df_2[(df_2['Year'] > 2004) & (df_2['Year'] <= 2024)] #narrow down search by year
df_4 = df_3.loc[df_3['popularity'] > 5000] #narrow down search to only include titles with relevant popularity
df_4['idMal'] = df_4['idMal'].astype(int) #change float to int
df_4['genres'] = df_4['genres'].apply(literal_eval) #convert string to list
df_4 = df_4.explode('genres') 
df_5 = df_4.loc[df_4['genres'] == 'Mahou Shoujo'] #narrow down to specific genre
df_5

The MAL API reqires you to search each show individualy, you can do it by name, but luckily anilist will allow you to return the MAL id. Using this we are able to loop through the id's for the shows that we want. If you would like to gather data for a larger set of ids be warned that it seems to top out at around 150 shows and no reasonable amount of time between querys seems to help, so prepare for that. You will need to apply for your own CLIENT ID from the My Anime List website. I highly suggest looking through [MAL API Club](https://myanimelist.net/clubs.php?cid=13727) for pointers.

**If you would like to run the MAL API code you will need to provide a file called client_id.py with your own client id from myanimelist.com (see client_id.py.example for the format)**

In [1]:
try:
    from client_id import CLIENT_ID
except ImportError:
    import os
    CLIENT_ID = os.getenv("CLIENT_ID")  # Try to get it from an environment variable
    if not CLIENT_ID:
        raise ValueError("CLIENT_ID not found. Please define it in client_id.py or as an environment variable.")


In [ ]:
def compile_mal(url):
    response = requests.get(url, headers = {
    'X-MAL-CLIENT-ID': client_id.CLIENT_ID
    })
    response.raise_for_status()
    anime = response.json()
    response.close()
    df = pd.json_normalize([anime])
    return(df)

mal_list= pd.DataFrame()

for i in df_5['idMal']:
    url = 'https://api.myanimelist.net/v2/anime/' + str(i) + '?fields=rank,mean,num_list_users'
    temp = compile_mal(url)
    mal_list = pd.concat([mal_list,temp],axis=0)
    time.sleep(2)

mal_list

In [ ]:
mal_list.to_csv('MAL_list.csv', index=False) #Write to csv